# 1. 加上Img_path並合併所有資料

In [13]:
import pandas as pd

# === 處理 df1 ===
df1 = pd.read_excel("data_1.xlsx", header=1)

# 改欄位名稱 & 單位轉換
df1 = df1.rename(columns={"Total lean mass (g)": "Total lean mass (kg)"})
df1["Total lean mass (kg)"] = df1["Total lean mass (kg)"] / 1000

# 新增 Img_path 欄位 (這裡假設用 UID 當檔名，如果沒有 UID 可以換成 index)
df1["Img_path"] = df1["編號"].apply(lambda x: f"data/Image/data_1/{x}/{x}.dcm")

# === 處理 df2 ===
df2 = pd.read_excel("data_2.xlsx", header=1)
df2["Img_path"] = df2["編號"].apply(lambda x: f"data/Image/data_2/{x}/{x}.dcm")

# === 處理 df3 ===
df3 = pd.read_excel("data_3.xlsx", header=1)
df3["Img_path"] = df3["編號"].apply(lambda x: f"data/Image/data_3/{x}/{x}.dcm")

# === 處理 df4 ===
df4 = pd.read_excel("data_4.xlsx", header=1)
df4["Img_path"] = df4["編號"].apply(lambda x: f"data/Image/data_4/{x}/{x}.dcm")

# === 合併 ===
merged_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [14]:
# 在合併的原始資料表最前面新增 UID 欄位以便後續追逤
merged_df.insert(
    0,  # 插在第一欄
    "UID",
    [f"U{str(i+1).zfill(4)}" for i in range(len(merged_df))]
)

In [15]:
# 取前四個欄位做對照表
mapping_df = merged_df.iloc[:, :4]

# 存成 CSV
mapping_df.to_csv("uid_mapping.csv", index=False, encoding="utf-8-sig")

print("對照表已輸出為 uid_mapping.csv")

對照表已輸出為 uid_mapping.csv


# 2. 保留有興趣欄位和處理NA和不存在的DICOM檔

In [16]:
# 只保留有興趣的column
merged_df = merged_df.drop(columns=['BMD+BMI(1)', '病歷號', '姓名', '1.Pelvis\n(THR)\n2.Pelvis\n(Routine)', 'T-score', 'Total lean mass (kg)', 'Total Fat(%fat)', 'leg lean mass(g)', 'leg Fat(%fat)'] )

In [17]:
# 找出有缺值的 row
rows_with_nan = merged_df[merged_df.isnull().any(axis=1)]

# 看看有幾列
print(f"共有 {len(rows_with_nan)} 列含有空值")

# 看看是哪些row有空值
print(rows_with_nan.head(30))

共有 24 列含有空值
       UID   編號  AGE  Gender(M:0,F:1)  檢查日期間格 XRAY日期 DXA日期  身高cm  體重kg  BMI  \
73   U0074   74  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
122  U0123  123  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
136  U0137  137  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
157  U0158  158  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
175  U0176  176  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
181  U0182  182  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
190  U0191  191  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
204  U0205  205  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
207  U0208  208  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
227  U0228  228  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
231  U0232  232  NaN              NaN     NaN    NaT   NaT   NaN   NaN  NaN   
269  U0270  270  NaN              NaN   

In [18]:
# Drop掉有空值的欄位
cleaned_df = merged_df.dropna()

print(f"原本 {len(merged_df)} 列，刪掉後剩 {len(cleaned_df)} 列")

原本 1300 列，刪掉後剩 1276 列


In [19]:
import os

# 找出不存在的檔案
missing_files = cleaned_df[~cleaned_df["Img_path"].apply(os.path.exists)]

print(f"共有 {len(missing_files)} 個檔案不存在！")
print(missing_files["Img_path"].tolist())  # 直接列出所有不存在的檔案路徑

共有 1 個檔案不存在！
['data/Image/data_4/684/684.dcm']


In [20]:
# Drop掉不存在的DICOM檔的row
all_data = cleaned_df[cleaned_df["Img_path"].apply(os.path.exists)]

In [21]:
all_data

,UID,編號,AGE,"Gender(M:0,F:1)",檢查日期間格,XRAY日期,DXA日期,身高cm,體重kg,BMI,ASMI(kg/m2),"Low muscle mass (yes 1, no 0)",Img_path
0,U0001,1,88.0,1.0,28.0,2017-12-22,2017-11-24,156.0,35.0,14.381986,4.520464,1.0,data/Image/data_1/1/1.dcm
1,U0002,2,75.0,1.0,34.0,2018-01-02,2017-11-29,149.2,50.8,22.820548,1.776678,1.0,data/Image/data_1/2/2.dcm
2,U0003,3,93.0,1.0,12.0,2017-12-13,2017-12-01,140.0,38.0,19.387755,3.975510,1.0,data/Image/data_1/3/3.dcm
3,U0004,4,92.0,1.0,28.0,2018-01-05,2018-02-02,141.9,46.4,23.043752,6.033589,0.0,data/Image/data_1/4/4.dcm
4,U0005,5,63.0,0.0,39.0,2018-01-16,2017-12-08,160.0,70.0,27.343750,8.808984,0.0,data/Image/data_1/5/5.dcm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,U1296,708,76.0,1.0,27.0,2022-11-11,2022-12-08,141.1,45.5,22.900000,5.050000,1.0,data/Image/data_4/708/708.dcm
1296,U1297,709,85.0,0.0,27.0,2022-11-11,2022-12-08,150.4,56.6,25.000000,6.180000,1.0,data/Image/data_4/709/709.dcm
1297,U1298,710,90.0,0.0,12.0,2022-12-18,2022-12-06,165.0,55.0,20.200000,6.410000,1.0,data/Image/data_4/710/710.dcm
1298,U1299,711,86.0,0.0,1.0,2022-12-01,2022-12-02,172.0,75.0,25.400000,6.360000,1.0,data/Image/data_4/711/711.dcm


# 3.重新命名欄位並切分train, test資料集

In [22]:
# 重新命名column
all_data.columns = [
    "UID", "No.", "Age", "Gender", "Time_period", "XRAY", "DXA", "Height", "Weight", "BMI", 
    "ASMI", "Low_muscle_mass", "Img_path"
]


In [23]:
# Drop掉所有time_period > 90的資料
# 記錄原始總筆數
n_before = len(all_data)

# 找出要 drop 的 row
to_drop = all_data[all_data["Time_period"] > 90]

# 實際 drop
final_data = all_data[all_data["Time_period"] <= 90]

# 記錄剩下的總筆數
n_after = len(final_data)

# 計算被刪掉的筆數
n_dropped = n_before - n_after

print(f"原始資料筆數: {n_before}")
print(f"刪掉筆數: {n_dropped}")
print(f"目前剩餘筆數: {n_after}")
print("被刪掉的 UID：")
print(to_drop["UID"].tolist())


原始資料筆數: 1275
刪掉筆數: 7
目前剩餘筆數: 1268
被刪掉的 UID：
['U0246', 'U0761', 'U1010', 'U1026', 'U1029', 'U1050', 'U1066']


In [29]:
# 存成csv檔
final_data.to_csv('final_data.csv', index=False)

In [31]:
import pandas as pd

# 假設 all_data 是你的完整 DataFrame
n = len(final_data)
split_idx = int(n * 0.9)

# 前 90% → development set
dev_set = final_data.iloc[:split_idx, :]

# 後 10% → external set
external_set = final_data.iloc[split_idx:, :]

print(f"總共 {n} 筆資料")
print(f"development set: {len(dev_set)} 筆")
print(f"external set: {len(external_set)} 筆")


總共 1268 筆資料
development set: 1141 筆
external set: 127 筆


In [32]:
# 存成csv檔
dev_set.to_csv("train.csv", index=False, encoding="utf-8-sig")
external_set.to_csv("test.csv", index=False, encoding="utf-8-sig")